In [4]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from transformers import pipeline
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [5]:
service = Service("chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service)

url = 'https://asunnot.oikotie.fi/myytavat-asunnot?pagination=1&locations=%5B%5B359,6,%22Sipoo%22%5D,%5B147,6,%22Kirkkonummi%22%5D%5D&buildingType%5B%5D=2&cardType=100'
driver.get(url)

In [18]:
products = driver.find_elements(by="xpath", value='//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]')

In [19]:
Plats = []
pris_storlke = []
Ovrigt = []

In [20]:

def click_next_page_button(driver):
    try:
        # Wait until the button with analytics-click-label="right" is present and clickable
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '[analytics-click-label="right"]'))
        )
        # Click the button
        button.click()
        print("Button clicked successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [21]:

def scrape_page(driver):
    try:
        # Extract product information
        products = driver.find_elements(By.XPATH, '//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]')

        for product in products:
            try:
                Plats.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__text card-v2-text-container__text--bold")]').text)
            except:
                Plats.append(None)  # Append None if the element is not found

            try:
                pris_storlke.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__group card-v2-text-container__group--boxed")]').text)
            except:
                pris_storlke.append(None)

            try:
                Ovrigt.append(product.find_element(By.XPATH, './/div[contains(@class, "card-v2-text-container__group card-v2-text-container__group--separated")]').text)
            except:
                Ovrigt.append(None)

        return Plats, pris_storlke, Ovrigt
    except Exception as e:
        print(f"An error occurred while scraping the page: {e}")
        return [], [], []


In [22]:
def scrape_all_pages(driver):
    all_Plats = []
    all_pris_storlke = []
    all_Ovrigt = []

    while True:
        # Scrape the current page
        Plats, pris_storlke, Ovrigt = scrape_page(driver)
        all_Plats.extend(Plats)
        all_pris_storlke.extend(pris_storlke)
        all_Ovrigt.extend(Ovrigt)

        try:
            # Click the "Next Page" button and wait for the page to load
            click_next_page_button(driver)
            
            # Wait for the page content to be refreshed
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")]'))
            )
            print("Moved to the next page.")
        except Exception as e:
            print(f"Failed to move to the next page or load next page content: {e}")
            break

    # Process the collected data
    print(f"Plats: {all_Plats}")
    print(f"Pris Storleke: {all_pris_storlke}")
    print(f"Ovrigt: {all_Ovrigt}")

In [23]:
scrape_all_pages(driver)

Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
Button clicked successfully.
Moved to the next page.
An error occurred: Message: 

Moved to the next page.
An error occurred: Message: 

Moved to the next page.
An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF68CAD9642+30946]
	(No symbol) [0x00007FF68CA8E3D9]
	(No symbol) [0x00007FF68C986FDA]
	(No symbol) [0x00007FF68C95CB85]
	(No symbol) [0x00007FF68CA037A7]
	(No symbol) [0x00007FF68CA1A771]
	(No symbol) [0x00007FF68C9FC813]
	(No symbol) [0x00007FF68C9CA6E5]
	(No symbol) [0x00007FF68C9CB021]
	GetHandleVerifier [0x00007FF68CC0F84D+1301229]
	GetHandleVerifier [0x00007FF68CC1BDC7+13517

In [25]:
df_plats = pd.DataFrame([x.split(', ') for x in Plats], columns=['Street Address', 'Area', 'City'])


In [26]:
df_plats

,Street Address,Area,City
0,Sundsberginkuja 1,Sundsberg,Kirkkonummi
1,Länsilenkki 12 C,Kuusala,Kirkkonummi
2,Männistöntie 1,Veikkola,Kirkkonummi
3,Sundetin puistotie 3 C,Sundsberg,Kirkkonummi
4,Impivaarantie 2 A,Veikkola,Kirkkonummi
...,...,...,...
151,Kyrkvallantie 1 C 9,Kyrkvalla,Kirkkonummi
152,Suursuontie 17,Nikkilä,Sipoo
153,Brontie 11 A,Bro,Kirkkonummi
154,Wiikintie 8 B,Keskusta,Kirkkonummi


In [29]:
pris_storlke

['50 552 €\n84,5 m²',
 '269 000 €\n64 m²',
 '149 000 €\n61 m²',
 '379 000 €\n112 m²',
 '167 000 €\n72 m²',
 '185 000 €\n80 m²',
 '229 000 €\n84 m²',
 '298 000 €\n68,5 m²',
 '295 000 €\n92 m²',
 '175 000 €\n74 m²',
 '389 900 €\n105 m²',
 '339 900 €\n91,5 m²',
 '339 900 €\n91,5 m²',
 '379 900 €\n107,5 m²',
 '198 000 €\n60,5 m²',
 '299 000 €\n98 m²',
 '319 000 €\n77,5 m²',
 '384 900 €\n105 m²',
 '195 000 €\n80 m²',
 '279 000 €\n83 m²',
 '349 000 €\n102 m²',
 '345 000 €\n68 m²',
 '208 000 €\n82,5 m²',
 '33 174 €\n84,5 m²',
 '375 000 €\n106 m²',
 '259 000 €\n119 m²',
 '32 225 €\n83 m²',
 '36 419 €\n93 m²',
 '153 000 €\n74 m²',
 '48 648 €\n82 m²',
 '185 000 €\n83,5 m²',
 '198 000 €\n79,5 m²',
 '354 000 €\n150 m²',
 '184 000 €\n83 m²',
 '339 900 €\n91,5 m²',
 '299 900 €\n79 m²',
 '279 900 €\n68 m²',
 '379 900 €\n107,5 m²',
 '155 000 €\n48 m²',
 '174 000 €\n114 m²',
 '69 588 €\n104 m²',
 '39 893 €\n75,5 m²',
 '318 000 €\n98,5 m²',
 '389 000 €\n89,5 m²',
 '28 924 €\n82 m²',
 '205 000 €\n80 m²',

In [31]:
# Splitting the list based on new lines
split_data = [x.split('\n') for x in pris_storlke]

# Determine the maximum number of parts
max_parts = max(len(item) for item in split_data)

# Create a DataFrame with columns for each part, filling missing values with NaN
columns = ['Price', 'Size', 'Maintenance', 'Price_per_sqm', 'Floors', 'Other']
#columns = ['Price', 'Size']

df_pris_storlk = pd.DataFrame([item + [np.nan] * (max_parts - len(item)) for item in split_data], columns=columns)


In [32]:
# Initialize lists to hold the split components
types = []
years = []
details = []

# Loop through the Ovrigt list and split each item
for item in Ovrigt:
    # Split on the first newline character
    parts = item.split('\n', 1)
    
    # Further split the first part to get the type and year
    type_year = parts[0].split(', ')
    
    # Assign values to type, year, and details
    type_val = type_year[0] if len(type_year) > 0 else None
    year_val = type_year[1] if len(type_year) > 1 else None
    detail_val = parts[1] if len(parts) > 1 else None
    
    # Append the values to the respective lists
    types.append(type_val)
    years.append(year_val)
    details.append(detail_val)

# Creating a DataFrame with columns for 'Type', 'Year', and 'Details'
df_ovrigt = pd.DataFrame({
    'Type': types,
    'Year': years,
    'Details': details
})



In [33]:
combined_df = pd.concat([df_plats, df_pris_storlk, df_ovrigt], axis=1)


In [34]:
combined_df

,Street Address,Area,City,Price,Size,Type,Year,Details
0,Sundsberginkuja 1,Sundsberg,Kirkkonummi,50 552 €,"84,5 m²",Rivitalo,2018,4h+kt+s
1,Länsilenkki 12 C,Kuusala,Kirkkonummi,269 000 €,64 m²,Rivitalo,2017,3h+k+kph+s
2,Männistöntie 1,Veikkola,Kirkkonummi,149 000 €,61 m²,Rivitalo,1977,2h+avok+kph+s+erill.wc+vh+ter+piha+v
3,Sundetin puistotie 3 C,Sundsberg,Kirkkonummi,379 000 €,112 m²,Rivitalo,2006,4h+k+s+2 terassia + 2 auton katos (yht.järj. m...
4,Impivaarantie 2 A,Veikkola,Kirkkonummi,167 000 €,72 m²,Rivitalo,1970,3h+k+kph
...,...,...,...,...,...,...,...,...
151,Kyrkvallantie 1 C 9,Kyrkvalla,Kirkkonummi,215 000 €,"83,5 m²",Rivitalo,1974,3h + s+k
152,Suursuontie 17,Nikkilä,Sipoo,159 000 €,74 m²,Rivitalo,1979,3h+k+s
153,Brontie 11 A,Bro,Kirkkonummi,229 000 €,"96,5 m²",Rivitalo,2004,"4h,k,s"
154,Wiikintie 8 B,Keskusta,Kirkkonummi,95 259 €,"30,5 m²",Rivitalo,1976,1h + kk + p


In [36]:
combined_df.to_excel('Utdata/Radhus_Kyrkslätt_sibo_18_08_2024.xlsx', index=False)


In [ ]:
## ot-card-v2__info-container padded padded--v20-h20    Lådan
#//div[contains(@class, "ot-card-v2__info-container padded padded--v20-h2")] Lådan
#card-v2-text-container__column--desktop-wide,  priset 
# //div[contains(@class, "card-v2-text-container__column--desktop-wide")]  priset



#//div[contains(@class, "card-v2-text-container__text card-v2-text-container__text--bold ")] Plats/ID


#"card-v2-text-container__column" Km2
# //div[contains(@class, "card-v2-text-container__column")]




